# 转置卷积

In [1]:
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

In [2]:
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h -1, X.shape[1] + w -1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i:i+h, j:j+w] += X[i,j]*K
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [5]:
X, K = X.reshape(1,1,2,2),K.reshape(1,1,2,2)
# bias=True表示转置卷积中会有一个可学习的偏置项
tconv = nn.ConvTranspose2d(1,1,kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

In [11]:
tconv = nn.ConvTranspose2d(1,1,kernel_size=2, padding=1,bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

In [12]:
tconv = nn.ConvTranspose2d(1,1,kernel_size=2, padding=1,stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 2.],
          [2., 0.]]]], grad_fn=<ConvolutionBackward0>)

In [14]:
X = torch.rand(size=(1,10,16,16))
conv = nn.Conv2d(10,20,kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

In [15]:
X = torch.arange(9.0).reshape(3,3)
K = torch.tensor([[1,2],[3,4]])
Y = d2l.corr2d(X,K)
Y

tensor([[27., 37.],
        [57., 67.]])

In [16]:
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [17]:
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [18]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])